In [34]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
import os

In [35]:
tool = TavilySearchResults(
    max_results=2
) # 2 results per query
print (type(tool))
print (tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [36]:
# AgentState 类定义了代理的状态数据结构
# 这是一个 TypedDict 类型的类,用于定义具有类型提示的字典
# 包含一个 messages 字段:
# - messages 字段是一个列表,存储 AnyMessage 类型的消息
# - 使用 Annotated 和 operator.add 标注,表示该字段支持列表拼接操作
# - AnyMessage 是一个联合类型,可以是各种消息类型(AIMessage、HumanMessage等)
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

我们需要一个函数来调用`Ollama`，一个函数来检查是否存在某种action，以及一个执行action的函数。

In [37]:
class Agent:
    def __init__ (self, model, tools, system=""):
        """
        初始化Agent实例
        参数:
            model: 要使用的语言模型,已配置好可以执行工具调用
            tools: 可供Agent使用的工具列表
            system: 可选的系统提示,用于指导模型行为
        功能:
            - 创建一个StateGraph来管理Agent的状态转换
            - 设置节点和边来定义状态转换流程:
              * llm节点: 调用LLM模型
              * action节点: 执行工具操作
            - 构建一个工具字典,用名称索引工具
            - 将工具绑定到模型
        """
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_ollama)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
    
    def exists_action(self, state: AgentState):
        """
        检查当前状态中是否存在需要执行的工具调用
        参数:
            state (AgentState): 当前代理状态
        返回:
            bool: 如果最新消息中包含工具调用则返回True,否则返回False
        功能:
            - 检查状态中最后一条消息是否包含工具调用
            - 决定是否需要转到action节点或结束流程
        """
        result = state['messages'][-1]
        if hasattr(result, 'tool_calls') and result.tool_calls:
            return True
        else:
            return False

    def call_ollama(self, state: AgentState):
        """
        调用Ollama模型处理当前消息流
        参数:
            state (AgentState): 当前代理状态,包含消息历史
        返回:
            dict: 包含模型响应的新状态更新
        功能:
            - 从状态中提取消息历史
            - 如果有system提示,则添加到消息中
            - 调用底层语言模型获取响应
            - 返回模型响应作为新消息
        """
        messages = state["messages"]
        if self.system:
            messages = state['messages']
        message = self.model.invoke(messages)
        return {'message': [message]}
    
    def take_action(self, state: AgentState):
        """
        执行模型请求的工具调用
        参数:
            state (AgentState): 当前代理状态
        返回:
            dict: 包含工具执行结果的新状态更新
        功能:
            - 从最新的消息中提取所有工具调用请求
            - 对每个工具调用:
              * 打印调用信息
              * 使用相应的工具执行操作
              * 将结果封装为ToolMessage
            - 返回所有工具调用结果作为新消息列表
        """
        # 找到消息列表的最后一个消息，而且一般来说此时的最后一个消息是一个tool call，由于可能出现并行工具或并行函数调用，所以这里可能是一个tool列表
        tool_calls = state['messages'][-1].tool_calls   
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], result=result))
        print("Back to the model!")
        return {'messages': results}

In [38]:
prompt = """You are a smart research assistant. Use the search engine to look up information.  \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOllama(
    model="qwen2.5:7b",
    temperature=0
)
abot = Agent(model, [tool], system=prompt)

In [41]:
messages = [HumanMessage(content="What is the weather in sf?")] # 之所以要用列表，因为代理期望使用的状态具有消息属性，这是一个消息列表
result = abot.graph.invoke({'messages': messages})
print(result)

{'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={})]}
